Thymio mini project

In [1]:
!pip install --upgrade tdmclient

In [2]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import extended_Kalman_filter as eKf
import vision
import geometry as geo


-1.5707963267948966


In [3]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node df054775-a7a5-454e-9ad3-7896a54bbcc8

In [4]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.left.target": [0],
            "motor.right.target": [0],
        })

async def move_robot(left_speed, right_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - left_speed    : left wheel speed in [thymio speed]
               - right_speed   : right wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.left.target": [left_speed],
            "motor.right.target": [right_speed],
        })

In [5]:
# Get the proximity sensor values
async def get_prox_sensors():
    """
    Get the sensor measurements of a the proximity sensors.
    Returns: - prox_sensors    : numpy array of fron proximity sensors measurements
             - detected        : boolean, true if any sensors measured a higher value than threshold
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 20) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    return prox_sensors, detected

# Get the motor speed measurement values
async def get_motor_speed_meas():
    """
    Get the motor speed measurements
    Returns: - speed_left_meas    : left wheel speed in [thymio speed]
             - speed_right_meas   : right wheel speed in [thymio speed]
    """
    
    await node.wait_for_variables({str('motor.left.speed')})
    speed_left_meas = node['motor.left.speed']
    await node.wait_for_variables({str('motor.right.speed')})
    speed_right_meas = node['motor.right.speed']

    return speed_left_meas, speed_right_meas

In [6]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d = await get_prox_sensors()
    print(y)
    print(d)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

[0 0 0 0 0]
False
[2270 3661 3363 5033 2548]
True
[2345 3726 3416 5083 2587]
True
[2341 3585 3160 4765 2211]
True
[2341 3585 3160 4765 2211]
True
[   0 2472    0    0    0]
True
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[   0    0    0 2117    0]
True
[   0    0    0 2388    0]
True
[   0    0    0 2548    0]
True
[   0    0    0 2659    0]
True


In [ ]:
#test vision and global nav
import vision
import global_navigation as gn

obstacles, goal_centroid = vision.vision_obstacles_and_goal()
robot_centroid, robot_direction = vision.vision_robot()

obstacles = obstacles.tolist()

merge_need = False
visualize = True
shortest_path, obstacles = gn.global_navigation(tuple(robot_centroid), obstacles, tuple(goal_centroid), merge_need, visualize)
print("glob nav path :", shortest_path)

obstacles.append([(4.5,3.5),(4.5,4.5),(6.5,4.5),(6.5,3.5)])
merge_need = True
visualize = True
shortest_path, obstacles = gn.global_navigation(tuple(robot_centroid), obstacles, tuple(goal_centroid), merge_need, visualize)
print("glob nav path :", shortest_path)

obstacles = np.array(obstacles)

In [9]:

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

robot, robot_direction = vision.vision_robot()
robot_orientation = np.arctan2(robot_direction[0], robot_direction[1])
robot = robot * 0.308

intermediate_goals = [(0, 10), (10, 10), (10, 0), (0,0)]
#robot = [0.0,0.0]
#robot_orientation = 0
robot_speed = [0.0,0.0]


Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    #prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    #robot[0] = robot[0] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.cos(robot_orientation)
    #robot[1] = robot[1] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.sin(robot_orientation)
    #robot_orientation = robot_orientation + dt * (robot_speed[0] - robot_speed[1])/(4.0*cst.WHEELS_DIST)
    #print(robot[0]," ", robot[1]," ", robot_orientation)
    
    Kfilter.dt_update(dt)
    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    Kfilter.measurement_wheels(speed_left_meas / 18.1818, speed_right_meas / 18.1818)
    
    if(iter % 10 == 0):
        robot_centroid, robot_direction = vision.vision_robot()
        robot_centroid = robot_centroid * 0.308
        robot_angle = np.arctan2(robot_direction[0], robot_direction[1])
        Kfilter.measurement_position(robot_centroid[0], robot_centroid[1], robot_angle)
    Kfilter.Kalman_filter()
    print("Kalman: ", Kfilter.Mu)

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation")
        #print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        if(angle > 0):
            robot_speed = [-30, 30]
        else:
            robot_speed = [30, -30]
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        print("translation ", np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2))
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 5:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        robot_speed = [70, 70]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        
        global_state = 'rotation'
    time.sleep(0.05)

    iter = iter + 1

0
Vecteur direction du robot : -2147483648
Centroid du robot : [-2147483648 -2147483648]
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.57082251e+00]
 [ 8.52501877e-01]
 [-7.97501756e-01]]
rotation
1
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-2.25268294e-01]
 [ 1.19900189e+00]
 [-1.14400181e+00]]
rotation
2
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.71798211e-01]
 [ 1.34115555e+00]
 [-1.28615549e+00]]
rotation
3
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.28610619e-01]
 [ 1.46397227e+00]
 [-1.47691350e+00]]
rotation
4
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-9.28502110e-02]
 [ 1.51095668e+00]
 [-1.48191170e+00]]
rotation
5
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-5.19292309e-02]
 [ 1.59689011e+00]
 [-1.92571041e+00]]
rotation
6
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [-1.23114413e-02]
 [ 1.66370666e+00]
 [-1.68732953e+00]]
rotation
7
Kalman:  [[-1.43167963e+09]
 [-1.43167963e+09]
 [ 2.58333335e-02]
 [ 1.68922841e+00]
 [-1.59627579e

CancelledError: 

In [31]:
await stop_robot()

In [11]:
import local_navigation as ln

#test control robot + kalman + local nav

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

intermediate_goals = [(0, 1)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]
speed_correction = 1


Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    Kfilter.dt_update(dt)
    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    speed_left_meas = speed_left_meas * speed_correction
    speed_right_meas = speed_right_meas * speed_correction
    Kfilter.measurement_wheels(speed_left_meas / 18.1818, speed_right_meas / 18.1818)
    Kfilter.Kalman_filter()
    print("Kalman: ", np.degrees(Kfilter.Mu[2]))
    

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation")
        #print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        speed_correction = 20.8
        if(angle > 0):
            robot_speed = [-20, 20]
        else:
            robot_speed = [20, -20]
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        print("translation ", np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2))
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 2:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        speed_correction = 0.65
        robot_speed = [50, 50]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        robot_direction = (1,0)
        # Object = ln.local_nav([Kfilter.Mu[0], Kfilter.Mu[1]], robot_direction, prox_sensors)
        object_robot_coords = ln.object_robot_coordinates(prox_sensors)
        print(object_robot_coords)
        print([Kfilter.Mu[0], Kfilter.Mu[1]])
        object_ground_coords = ln.robot_to_ground_coords([Kfilter.Mu[0], Kfilter.Mu[1]], robot_direction, object_robot_coords)
        print(object_ground_coords)
        Object = ln.safety_zone(object_ground_coords)
        print(iter)
        print(Object)
        await stop_robot()
        break
        #global_state = 'rotation'

    time.sleep(0.05)

    iter = iter + 1

0
rotation
Kalman:  [0.]
1
rotation
Kalman:  [0.79955218]
2
rotation
Kalman:  [1.78651747]
3
rotation
Kalman:  [2.8061579]
4
rotation
Kalman:  [3.86725701]
5
rotation
Kalman:  [4.92111137]
6
rotation
Kalman:  [5.94677408]
7
rotation
Kalman:  [6.96811438]
8
rotation
Kalman:  [7.99970148]
9
rotation
Kalman:  [9.02894333]
10
rotation
Kalman:  [10.12858578]
11
rotation
Kalman:  [11.36485993]
12
rotation
Kalman:  [12.70996226]
13
rotation
Kalman:  [13.96108468]
14
rotation
Kalman:  [15.04212739]
15
rotation
Kalman:  [16.03865396]
16
rotation
Kalman:  [17.11601316]
17
rotation
Kalman:  [18.13889388]
18
rotation
Kalman:  [19.09205213]
19
rotation
Kalman:  [20.12346676]
20
rotation
Kalman:  [21.19349392]
21
rotation
Kalman:  [22.17432169]
22
rotation
Kalman:  [23.15296523]
23
rotation
Kalman:  [24.09721523]
24
rotation
Kalman:  [25.22377135]
25
rotation
Kalman:  [26.47225121]
26
rotation
Kalman:  [27.40003442]
27
rotation
Kalman:  [28.10778706]
28
rotation
Kalman:  [28.89254321]
29
rotation
Ka

In [9]:
await stop_robot()

In [16]:
#CODE TO CALIBRATE ROTATION
speed_correction = 12.5

intermediate_goals = [(0, 1)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]
speed_correction = 1

Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)

robot_speed = [-30, 30]
await move_robot(robot_speed[0], robot_speed[1])

old_time = 0.0

iter = 0
while(True):
    iter = iter + 1
    print(iter)

    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    speed_left_meas = speed_left_meas * speed_correction
    speed_right_meas = speed_right_meas * speed_correction
    Kfilter.measurement_wheels(speed_left_meas / 18.1818, speed_right_meas / 18.1818)

    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    old_time = time.time()

    Kfilter.dt_update(dt)
    Kfilter.Kalman_filter()
    print(dt, time.time(), old_time)
    print("Kalman: ", Kfilter.Mu[2][0])

    if(abs(Kfilter.Mu[2][0]) > np.pi / 4.0):
        await stop_robot()
        print("stop")
        break

    await client.sleep(0.2)

1
1701552189.3022761 1701552189.3022761 1701552189.3022761
Kalman:  -5.149876337894279e-08
2
0.20335006713867188 1701552189.5056262 1701552189.5056262
Kalman:  -0.012477136474676104
3
0.20076799392700195 1701552189.7063942 1701552189.7063942
Kalman:  -0.031536374467850146
4
0.20118427276611328 1701552189.9075785 1701552189.9075785
Kalman:  -0.0536010863690173
5
0.20019984245300293 1701552190.1077783 1701552190.1077783
Kalman:  -0.07359592660971309
6
0.20142674446105957 1701552190.309205 1701552190.309205
Kalman:  -0.0952223287572143
7
0.20737385749816895 1701552190.5181653 1701552190.516579
Kalman:  -0.11871897487167656
8
0.20075321197509766 1701552190.7173321 1701552190.7173321
Kalman:  -0.14050379686941292
9
0.20030593872070312 1701552190.919734 1701552190.917638
Kalman:  -0.16218674808691516
10
0.21271443367004395 1701552191.133277 1701552191.1303525
Kalman:  -0.18540948720346143
11
0.20468688011169434 1701552191.3350394 1701552191.3350394
Kalman:  -0.2077386745701087
12
0.200441122

In [28]:
await stop_robot()